# Monte Carlo Simulation for Option Pricing

We aim to price **European and Asian options** using a Monte Carlo simulation. The method consists of the following steps:

### Step 1: Simulate multiple trajectories of the underlying asset price  
Using Geometric Brownian Motion (GBM), we generate different price paths for the underlying asset over time.

### Step 2: Compute the option payoff for each simulation  
- **European options:**  
  - **Call option:** $ \max(S_T - K, 0) $  
  - **Put option:** $ \max(K - S_T, 0) $  
  where $ S_T $ is the asset price at maturity and $ K $ is the strike price.  

- **Asian options:**  
  - **Call option:** $ \max(\text{mean}(S_t) - K, 0) $  
  - **Put option:** $ \max(K - \text{mean}(S_t), 0) $  
  where $ \text{mean}(S_t) $ is the average asset price over the option's lifetime.

### Step 3: Compute the expected payoff  
The option price is approximated by averaging the payoffs across all simulations.

### Step 4: Discount the expected payoff  
To obtain the present value of the option, we discount the expected payoff using the risk-free rate $ r_f $:

$$
\text{Option Price} = e^{-r_f T} \cdot \mathbb{E}[\text{Payoff}]
$$

---

## Step 1: Simulating the Underlying Asset Price

To model the asset price dynamics, we use the **Geometric Brownian Motion (GBM)**, a standard stochastic process in finance.

### Variance Reduction using Antithetic Variables

To speed up the convergence of our Monte Carlo simulation, we use a variance reduction technique called **antithetic variables**.    
Specifically, we geneate a set of random variables and then create their opposite counterparts (i.e., fo a variable $ Zt $, we also generate $ -Zt $).     
This "mirroring" helps stabilize the payoff estimations, making the convergence towards the true option price faster and more efficient.

### Vectorized Approach with NumPy

Instead of using loops, we employ a **vectorized approach** with `NumPy` for efficiency.  
Each price $ S_t $ depends on the previous price $ S_{t-1} $. Using NumPy's `cumprod` function, we can apply the exponential growth factor iteratively.

### Discretized GBM Formula

The asset price follows the **discretized Black-Scholes equation**:

$$
S_{t+1} = S_t \cdot \exp\left( \left( \mu - \frac{1}{2} \sigma^2 \right) \Delta t + \sigma \sqrt{\Delta t} Z_t \right)
$$

where:
- $ S_t $ is the asset price at time $ t $,
- $ \mu $ is the drift (expected return),
- $ \sigma $ is the asset’s volatility,
- $ \Delta t $ is the time step ($ \frac{1}{252} $ for daily simulation),
- $ Z_t \sim N(0,1) $ is a standard normal random variable.

### Reformulation as a Cumulative Product

By iterating over time, the asset price can be rewritten as a **cumulative product**:

$$
S_{t+1} = S_0 \cdot \prod_{i=1}^{t} \exp\left( \left( \mu - \frac{1}{2} \sigma^2 \right) \Delta t + \sigma \sqrt{\Delta t} Z_i \right)
$$

where:
- $ S_0 $ is the initial asset price,
- $ S_{t+1} $ is the price at time $ t+1 $,
- $ Z_i \sim N(0,1) $ are independent normal variables.

This formulation enables the simulation of multiple price paths efficiently using **matrix operations**.


## 1. Setting Up the Environment

In [ ]:
import numpy as np
from scipy.stats import norm # For the Black & Scholes method

## 2. Inputs

In [108]:
S0 = 100      # Initial price of the underlying asset
K = 90        # Option strike price
T = 0.5         # Time to maturity in years
rf = 0.02     # Annual risk-free rate
sigma = 0.2   # Annualized volatility of the underlying asset
n_simul = 100000  # Number of Monte Carlo simulations
dt = 1 / 252  # Time step: 252 for daily, 12 for monthly, 52 for weekly...
option_type = 'call'  # Type of European option
option_nature = 'european'  # Option nature: 'european' or 'asiatic'


## 3. Input Validation

In [109]:
assert isinstance(S0, (int, float)), "S0 must be a number (int or float)"
assert isinstance(K, (int, float)), "K must be a number (int or float)"
assert isinstance(T, (int, float)), "T must be a number (int or float)"
assert isinstance(rf, (int, float)), "rf must be a number (int or float)"
assert isinstance(sigma, (int, float)), "sigma must be a number (int or float)"
assert isinstance(n_simul, int), "n_simul must be an integer"
assert isinstance(dt, (int, float)), "dt must be a number (int or float)"
assert option_type in ['call', 'put'], "option_type must be either 'call' or 'put'"
assert option_nature in ['european', 'asiatic'], "option_nature must be either 'european' or 'asiatic'"

## 4. Monte-Carlo simulation

In [123]:
# Calculation of the number of time steps
n_steps = int(T / dt)

# Generation of random variables following a standard normal distribution
# Each row corresponds to a simulation, and each column represents a time step
random_walks = np.random.normal(0, 1, (int(n_simul/2), n_steps))

# We use the antithetic variables method as a variance reduction technique
antithetic_random_walks = np.vstack((random_walks, -random_walks))

# Calculation of the multiplicative coefficients at each time step
# The price evolution formula follows the discretized Itô equation (Black-Scholes model)
coeff_multiplicateur = np.exp((rf - (sigma**2) / 2) * dt + sigma * (dt**0.5) * antithetic_random_walks)

# Simulation of underlying asset prices at each time step
simulated_prices = S0 * np.cumprod(coeff_multiplicateur, axis=1)

# Calculation of payoffs based on the option type (European or Asian)
if option_nature == "european":
    if option_type == 'call':
        # Payoff for a European call option: max(S_T - K, 0)
        payoff = np.maximum(simulated_prices[:, -1] - K, 0)
    elif option_type == 'put':
        # Payoff for a European put option: max(K - S_T, 0)
        payoff = np.maximum(K - simulated_prices[:, -1], 0)

elif option_nature == "asiatic":
    if option_type == 'call':
        # Payoff for an Asian call option: max(mean(S) - K, 0)
        payoff = np.maximum(np.mean(simulated_prices, axis=1) - K, 0)
    elif option_type == 'put':
        # Payoff for an Asian put option: max(K - mean(S), 0)
        payoff = np.maximum(K - np.mean(simulated_prices, axis=1), 0)

# Discounting the payoffs to obtain the option value
discount_factor = np.exp(-rf * T)
option_price = discount_factor * np.mean(payoff)

# Displaying the option price
print(f"Price of the {option_nature} {option_type} option (calculated with {n_simul} Monte-Carlo simulations): {option_price:.2f}")


Price of the european call option (calculated with 100000 Monte-Carlo simulations): 12.44


## 5. Comparing Monte-Carlo Results with the Black-Scholes Formula

For **European options**, we can compare the option price obtained via **Monte Carlo simulation** with the **Black-Scholes analytical formula**.

The Black-Scholes formula provides a theoretical price for European call and put options, given by:

$
C = S_0 N(d_1) - K e^{-r_f T} N(d_2)
$

$
P = K e^{-r_f T} N(-d_2) - S_0 N(-d_1)
$

where:

$
d_1 = \frac{\ln(S_0/K) + (r_f + \frac{1}{2} \sigma^2)T}{\sigma \sqrt{T}}, \quad d_2 = d_1 - \sigma \sqrt{T}
$

In [115]:
# /!\ To be used only for European options

# Black-Scholes function to calculate the value of a European option (call or put)
def black_scholes_option(S0, K, T, rf, sigma, option_type):
    # Calculating the d1 and d2 variables
    d1 = (np.log(S0 / K) + (rf + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        # Calculating the value of the call option
        option_value = S0 * norm.cdf(d1) - K * np.exp(-rf * T) * norm.cdf(d2)
    elif option_type == 'put':
        # Calculating the value of the put option
        option_value = K * np.exp(-rf * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    else:
        raise ValueError("Option type must be 'call' or 'put'")
    
    return option_value

# Calculating the option value based on type (call or put)
if option_nature != 'european':
    raise ValueError("The B&S function only works for European options")
else:
    option_price = black_scholes_option(S0, K, T, rf, sigma, option_type)
    print(f"Price of the European {option_type} option (calculated using the Black-Scholes formula): {option_price:.2f}")

Price of the European call option (calculated using the Black-Scholes formula): 12.45


## 6. Consistency of Monte Carlo Results with Black-Scholes

The results obtained from our **Monte Carlo simulation** for European options are **very close** to those computed with the **Black-Scholes formula**.  

This is expected, as the Monte Carlo method is simply a numerical approximation of the same theoretical model used in Black-Scholes. The slight discrepancies arise due to simulation noise, which decreases as we increase the number of simulations.
